In [17]:
import pandas as pd
import tweepy
import re
import nltk
import numpy as np
#from wordcloud import WordCloud, STOPWORDS

### Twitter APIv2
- Authenticate with Twitter

In [9]:
#Authenticate with Twitter
client = tweepy.Client(bearer_token = 'AAAAAAAAAAAAAAAAAAAAAAG0hwEAAAAAv43y4zBhJ4zrA97fpCeo%2FKuIFBU%3DxnrOe20bfF2UBohW36CfF2i4eXJZz1bDoOYIrQ6hpDaMJPUpzD')

#### Syntax for no retweets

In [14]:
# -is:retweet means no retweets
# query = '#metgala2022 -is:retweet lang:en'
# tweets = client.search_recent_tweets(query=query, tweet_fields = ['context_annotations', 'created_at'], max_results =10)

### Pull Tweets from Twitter

In [18]:
#tweets = client.search_recent_tweets(query=query, tweet_fields = ['context_annotations', 'created_at'], max_results =10)

- inputs below getting us 1000 tweets
- between 4,000 and 9,000 words
- the most common length of a tweet is 33 characters (aprox 4-9 words in 33 chart)

In [20]:
query = '#metgala2022 lang:en'
paginator = tweepy.Paginator(
    client.search_recent_tweets,           # The method you want to use
    query=query,                           # Some argument for this method
    max_results=100,                       # How many tweets per page
    limit=10                               # How many pages to retrieve
)
# Get tweets that contain the hashtag #TypeKeywordHere
# -is:retweet means I don't want retweets
# lang:en is asking for the tweets to be in english

tweet_list = []

for tweet in paginator.flatten(): # Total number of tweets to retrieve
    tweet_list.append(tweet)
    print(tweet)

RT @incorrectgl: Xiao Yi and Pei Shuangyi from ‘Led Astray By Love’ have arrived at the #MetGala  
#MetGala2022 https://t.co/nRgtPauaX4
RT @noonegifaker: Olivia Rodrigo #OliviaRodrigo #Metgala #MetGala2022 #exposed #deepfakes #fake #celebrityfake #celebrityleaks #pornedit #p…
RT @KangsterLiberal: #KanganaRanaut really understood her assignment 💅

Ate and left no crumbs! 👁️👄👁️

This is the #MetGala that y'all shou…
RT @Kristennetten: “My Mom wanted to come, so I brought my Mom” — @elonmusk✨ 💛✨attending the #MetGala2022 
🎥: @etnow https://t.co/87pnZDfb9W
RT @chrispinelive: Me and Dad 🍲📃🎱 #MetGala2022 #ChrisPine #Robertpine #LosAngeles https://t.co/2sg4dFCeus
RT @scr0nchy: shining Johnny! today was so bright because of your achievement. so so proud of you ❤️‍🔥 
#MetGala2022  #JOHNNYMetGala 
@NCTs…
RT @davemmrk: Thanks babe for pounding me so smoothly. #Pleasure #bottomboys #Arabiandick #MetGala2022 https://t.co/QWBOKA05eO
RT @TwitterFashion: they're giving spring #MetGala2022 https://t.co

### Create dataframe of tweets

In [21]:
tweet_list_df = pd.DataFrame(tweet_list)
tweet_list_df = pd.DataFrame(tweet_list_df['text'])
tweet_list_df.head(5)

,text
0,RT @incorrectgl: Xiao Yi and Pei Shuangyi from...
1,RT @noonegifaker: Olivia Rodrigo #OliviaRodrig...
2,RT @KangsterLiberal: #KanganaRanaut really und...
3,"RT @Kristennetten: “My Mom wanted to come, so ..."
4,RT @chrispinelive: Me and Dad 🍲📃🎱 #MetGala2022...


### Data Preprocessing
- sub() function that returns a string after replacing the matched pattern in a string with a replacement.
- Syntax:
re.sub(pattern, repl, string, count=0, flags=0) https://www.pythontutorial.net/python-regex/python-regex-sub/

In [22]:
def preprocess_tweet(sen):
    '''Cleans text data up, leaving only 2 or more char long non-stepwords composed of A-Z & a-z only
    in lowercase'''
    
    sentence = sen.lower()

    # Remove RT
    sentence = re.sub('RT @\w+: '," ", sentence)

    # Remove special characters
    sentence = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  

    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)  

    return sentence

In [23]:
cleaned_tweets = []

for tweet in tweet_list_df['text']:
  cleaned_tweet = preprocess_tweet(tweet)
  cleaned_tweets.append(cleaned_tweet)

In [31]:
tweet_list_df['cleaned'] = pd.DataFrame(cleaned_tweets)
tweet_list_df.head(50)

,text,cleaned
0,RT @incorrectgl: Xiao Yi and Pei Shuangyi from...,rt xiao yi and pei shuangyi from led astray by...
1,RT @noonegifaker: Olivia Rodrigo #OliviaRodrig...,rt olivia rodrigo oliviarodrigo metgala metgal...
2,RT @KangsterLiberal: #KanganaRanaut really und...,rt kanganaranaut really understood her assignm...
3,"RT @Kristennetten: “My Mom wanted to come, so ...",rt my mom wanted to come so brought my mom att...
4,RT @chrispinelive: Me and Dad 🍲📃🎱 #MetGala2022...,rt me and dad metgala2022 chrispine robertpine...
5,RT @scr0nchy: shining Johnny! today was so bri...,rt shining johnny today was so bright because ...
6,RT @davemmrk: Thanks babe for pounding me so s...,rt thanks babe for pounding me so smoothly ple...
7,RT @TwitterFashion: they're giving spring #Met...,rt they re giving spring metgala2022
8,RT @swiftoursonggg: The men in Met Gala every ...,rt the men in met gala every year metgala metg...
9,"RT @ckbdidit: gala met billie, and then met gu...",rt gala met billie and then met gunna metgala2...
